In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# reading in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
words[:8]

32033
15


['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# building the vocanulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s : i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# building the dataset

block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# utility function that will be used for comparing manual gradients to PyTorch gradient
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item() # checks if all true inside the tenor, here if condition true for every element will return true else false
    app = torch.allclose(dt, t.grad) # checks if both close numerically ie |dt - t.grad| ~ 0, will return true, else false
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embed = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # no. of neurons in hidden layer

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed),              generator = g)
# layer 1
W1 = torch.randn((n_embed*block_size, n_hidden),     generator=g) * (5/3)/((n_embed * block_size)**0.5)
b1 = torch.randn(n_hidden,     generator=g) * 0.1   # it's useless bcz of batch normalization layer

# layer 2
W2 = torch.randn((n_hidden, vocab_size),     generator=g) * 0.1
b2 = torch.randn(vocab_size,     generator=g) * 0.1   # it's useless bcz of batch normalization layer

# batchnorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note:initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect implementation of the backward pass.

parameters = [C, W1, W2, b1, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [8]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embedding characters into vectors
embcat = emb.view(emb.shape[0], -1)
# linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre activation
# batch norm layer
bnmeani = (1/n)*hprebn.sum(0, keepdim=True) # n is batch_size here
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True) # note: bessel's correction(dividing by n-1)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# non linearity
h = torch.tanh(hpreact) # hidden layer

# linear layer 2
logits = h@W2 + b2 # output layer

# cross entropy loss(same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3482, grad_fn=<NegBackward0>)

#### * Backprop through the whole thing manually
#### * Backpropagating through all the variables
#### * as they are defined in the forward pass above

In [9]:
logprobs.shape

torch.Size([32, 27])

In [10]:
logprobs[range(n), Yb] # 32 context vectors next probable character's index is in Yb
# so what the above expression gives is the probabilityb assigned by the model to the correct index for a context vector for the size of 32 vectors
# loss = -(a+b+c...(32))/32 here
# dloss/da = -1.0/32 here
# these 32 values will be covered but what about other remaining values left in logprobs(32, 27) as dlogprobs will also have the same shape as logprobs, they would be 0

tensor([-3.8865, -3.0544, -3.7539, -3.1745, -4.1875, -3.4588, -3.2328, -4.0198,
        -3.2284, -4.2556, -3.1185, -1.6730, -2.7523, -2.8566, -2.9019, -3.1699,
        -3.8521, -3.0951, -3.5424, -3.4971, -2.8946, -3.0708, -4.2797, -3.9976,
        -3.5286, -2.8956, -3.1412, -3.9343, -2.7687, -3.5159, -3.2771, -3.1273],
       grad_fn=<IndexBackward0>)

In [11]:
counts.shape, counts_sum_inv.shape # shapes are different(were broadcasted and then multiplied)

(torch.Size([32, 27]), torch.Size([32, 1]))

In [17]:
# dlogprobs will also have the same shape as logprobs, hence need to calculate loss like that 
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
dprobs = (1.0/probs) * dlogprobs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True) # need to take broadcasting into picture as well
                   # and how it is being taken into account is think it like this:
                   # already taken if used once for the 1st row, what about other rows
                   # their gradients will also come and add to the current gradient
                   # hence .sum(1(in dir of row), and since will be of same dimension as counts_sum_in)
dcounts = counts_sum_inv * dprobs
# can't compute as of now as also depends upon other parameters as well
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum # letting broadcasting take care
dnorm_logits = counts * dcounts
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
# max value operation is being operated there, giving a no. Therefore local derivative=1(only for those places).
# but need to multiply global derivative to those places only where max element present.
# so doing backprop using one hot vector
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1])*dlogit_maxes
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1-h**2) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv
dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar 
dbndiff += (2*bndiff) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-torch.ones_like(bndiff) * dbndiff).sum(0, keepdim=True)
dhprebn += (1.0/n) * (torch.ones_like(hprebn) * dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
# backprop of cocatenation (.view())
# just reinterpretting what it was before
demb = dembcat.view(emb.shape)
# the first line ---> emb = C[Xb]
# Xb here is the list of contexts that are provided to the model of size (32,3) as 32 contexts in total in a batch
# then we get the row of these characters by plucking them into C to get the embedding.
# now to backpropagate through this step. -> dC, need to bet the rows that were plucked out of C
# by looking at the indices of Xb and if repeating need to add these grads.
# to get indices of Xb(shape =(32, 3))
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        # index
        ix = Xb[k, j]
        # need to update the grad of this row
        dC[ix] += demb[k,j]
 

cmp('logprobs', dlogprobs, logprobs) # already ran the forward pass for a single batch and checking if our calculated gradient equal to torch calculatwd gradient
cmp('dprobs', dprobs, probs)
cmp('dcounts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('dcounts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('dlogit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('bnbias', dbnbias, bnbias)
cmp('bndiff', dbndiff, bndiff)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('hprebn', dhprebn, hprebn)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
dprobs          | exact: True  | approximate: True  | maxdiff: 0.0
dcounts_sum_inv | exact: True  | approximate: True  | maxdiff: 0.0
dcounts_sum     | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
dlogit_maxes    | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff:

In [13]:
dcounts_sum_inv.shape

torch.Size([32, 1])

In [19]:
# backpropagating through cross entropy but all in one go
# in the previous block we calculated loss by using big chunk of code which isn't required

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdim=True)
# counts_sum_inv = counts_sum**-1
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
fast_loss = F.cross_entropy(logits, Yb)
print('loss', fast_loss.item(), 'diff:', (fast_loss - loss).item())

loss 3.34820818901062 diff: -2.384185791015625e-07


In [20]:
# backward pass 

dlogits = F.softmax(logits, dim=1) # along the rows need to calculate probs
# if i = y ie the correct label need to subtract 1
dlogits[range(n), Yb] -= 1
# and take the mean
dlogits /= n

cmp('logits', dlogits, logits) # approximately = True

logits          | exact: False | approximate: True  | maxdiff: 6.05359673500061e-09


In [21]:
# now backpropagating through batchnormalization in 1 go

# before:
bnmeani = (1/n)*hprebn.sum(0, keepdim=True) # n is batch_size here
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True) # note: bessel's correction(dividing by n-1)-
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / torch.sqrt(hpreact.var(0, keepdim=True, unbiased=True)) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(3.2902e-05, grad_fn=<MaxBackward1>)


In [22]:
# backward pass of bactchnorm layer
# calculating dhprebn(which gets passed to be batchnormalised) and relating to dhpreact_fast(what we got out)

dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
